In [1]:
%reload_ext autoreload
%autoreload 2

from pytorch_lightning import seed_everything
from src.acnets.deep import WaveletModel, MaskedModel
from src.acnets.datasets import LEMONDataModule, Julia2018DataModule

seed_everything(42)

lemon_datamodule = LEMONDataModule(
    atlas='dosenbach2010', kind='partial correlation',
    aggregation_strategy='time_networks',
    test_ratio=.1, val_ratio=.05,
    segment_length=32,
    n_subjects=215, batch_size=32)
lemon_datamodule.setup()

julia2018_datamodule = Julia2018DataModule(
    atlas='dosenbach2010', kind='partial correlation',
    aggregation_strategy='time_networks',
    segment_length=32,
    test_ratio=.5, batch_size=32)
julia2018_datamodule.setup()

n_regions = julia2018_datamodule.train[0][0].shape[-1]

for n_embeddings in [4, 8, 16, 32, 64, 128, 256]:
    model = MaskedModel(n_regions, n_embeddings)

    trainer = model.fit(lemon_datamodule, max_epochs=2500, phase='pretrain')
    tuner = model.fit(julia2018_datamodule, max_epochs=2500, phase='finetune')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/S2SAE


ckpt_id: 2206f7


/home/morteza/miniforge3/envs/acnets/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /home/morteza/workspace/ACNets/checkpoints/S2SAE_2206f7 exists and is not empty.
Restoring states from the checkpoint path at /home/morteza/workspace/ACNets/checkpoints/S2SAE_2206f7/last.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type               | Params
---------------------------------------------------------
0 | accuracy          | MulticlassAccuracy | 0     
1 | feature_extractor | Seq2SeqAE          | 540   
2 | cls_head          | Classifier         | 36    
---------------------------------------------------------
576       Trainable params
0         Non-trainable params
576       Total params
0.002     Total estimated model params size (MB)
Restored all states from the checkpoint at /home/morteza/workspace/ACNets/checkpoints/S2SAE_2206f7/last.ckpt
`Trainer.fit` stopped: `max_epochs=2500` reached.
GPU availabl

ckpt_id: 7d1ff2


/home/morteza/miniforge3/envs/acnets/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /home/morteza/workspace/ACNets/checkpoints/S2SAE_7d1ff2 exists and is not empty.
Restoring states from the checkpoint path at /home/morteza/workspace/ACNets/checkpoints/S2SAE_7d1ff2/last.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type               | Params
---------------------------------------------------------
0 | accuracy          | MulticlassAccuracy | 0     
1 | feature_extractor | Seq2SeqAE          | 1.0 K 
2 | cls_head          | Classifier         | 118   
---------------------------------------------------------
1.1 K     Trainable params
0         Non-trainable params
1.1 K     Total params
0.004     Total estimated model params size (MB)
Restored all states from the checkpoint at /home/morteza/workspace/ACNets/checkpoints/S2SAE_7d1ff2/last.ckpt
/home/morteza/miniforge3/envs/acnets/lib/python3.11/site-packa